In [ ]:
# Notebook 02 – Feature Engineering cho Home Credit Default Risk
# Mục tiêu:
# - Tạo bộ feature hoàn chỉnh từ tất cả bảng: application, bureau, prev, pos, inst, credit_card
# - Không scaling, không WOE, không PCA

import pandas as pd
import numpy as np
import os
import gc
import warnings
from typing import Tuple

warnings.filterwarnings("ignore")

DATA_DIR = "/content/drive/MyDrive/home_credit"
OUTPUT_DIR = os.path.join(DATA_DIR, "processed")
os.makedirs(OUTPUT_DIR, exist_ok=True)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

def RELU(series: pd.Series) -> pd.Series:
    return series.apply(lambda x: max(0, x))

def safe_divide(numerator, denominator, fill_value=np.nan):
    """
    Chia an toàn:
    - tránh division by zero
    - có thể dùng cho Series hoặc ndarray
    """
    numerator = numerator.astype(float)
    denominator = denominator.astype(float)
    result = numerator / (denominator + 1e-9)
    if isinstance(result, pd.Series):
        mask = (denominator == 0) | denominator.isna()
        result[mask] = fill_value
    else:
        mask = (denominator == 0)
        result[mask] = fill_value
    return result


In [ ]:
# ============================================================================
# 1. APPLICATION TABLE - MAIN FEATURES
# ============================================================================

def process_application(df: pd.DataFrame) -> pd.DataFrame:
    """
    Xử lý bảng application_train/test
    - Anomaly handling
    - Flags & indicators
    - Ratios (income, credit, payment)
    - EXT_SOURCE features
    - Time transforms
    - Group-based features
    """
    df = df.copy()

    # -------------------------------------------------------------------------
    # 1.1. ANOMALY & MISSING HANDLING
    # -------------------------------------------------------------------------

    # DAYS_EMPLOYED anomaly
    df['FLAG_EMPLOYED_ANOMALY'] = (df['DAYS_EMPLOYED'] == 365243).astype(int)
    df.loc[df['DAYS_EMPLOYED'] == 365243, 'DAYS_EMPLOYED'] = np.nan

    # CODE_GENDER XNA
    df['FLAG_GENDER_XNA'] = (df['CODE_GENDER'] == 'XNA').astype(int)
    df['CODE_GENDER'] = df['CODE_GENDER'].replace('XNA', 'F')

    # Missing gradings indicator
    grading_cols = [col for col in df.columns if '_AVG' in col or '_MODE' in col]
    df['MISSING_GRADINGS'] = df[grading_cols].isna().sum(axis=1)

    # Clean categorical anomalies
    df['EMERGENCYSTATE_MODE'] = df['EMERGENCYSTATE_MODE'].fillna('No')
    df['OCCUPATION_TYPE'] = df['OCCUPATION_TYPE'].replace({
        'IT staff': 'High skill tech staff',
        'Realty agents': 'Sales staff'
    })

    # Simplify categories
    occupation_to_drop = ['High skill tech staff', 'HR staff', 'Secretaries',
                          'Cleaning staff', 'Cooking staff', 'Waiters/barmen staff',
                          'Private service staff', 'Low-skill Laborers', 'Security staff']
    df['OCCUPATION_TYPE'] = df['OCCUPATION_TYPE'].replace(occupation_to_drop, np.nan)

    df['FONDKAPREMONT_MODE'] = df['FONDKAPREMONT_MODE'].replace(
        ['not specified', 'org spec account', 'reg oper account'], np.nan
    )

    # Drop sparse columns
    df = df.drop(columns=['HOUSETYPE_MODE', 'WALLSMATERIAL_MODE'], errors='ignore')

    # -------------------------------------------------------------------------
    # 1.2. FLAGS & INDICATORS
    # -------------------------------------------------------------------------

    # Family structure
    df['cnt_non_child'] = df['CNT_FAM_MEMBERS'] - df['CNT_CHILDREN']
    df['FLAG_SINGLE'] = (df['CNT_FAM_MEMBERS'] == 1).astype(int)
    df['FLAG_HAS_CHILDREN'] = (df['CNT_CHILDREN'] > 0).astype(int)
    df['child_to_non_child_ratio'] = safe_divide(
        df['CNT_CHILDREN'],
        df['cnt_non_child']
    )

    # Address reliability
    df['RELIABILITY_IN_CUSTOMER_CITY'] = (
        df['REG_CITY_NOT_LIVE_CITY'] +
        df['REG_CITY_NOT_WORK_CITY'] +
        df['REG_REGION_NOT_LIVE_REGION'] +
        df['REG_REGION_NOT_WORK_REGION'] +
        df['LIVE_CITY_NOT_WORK_CITY'] +
        df['LIVE_REGION_NOT_WORK_REGION']
    )

    # Contact information
    df['SUM_CONTACTS'] = (
        df['FLAG_MOBIL'] + df['FLAG_EMP_PHONE'] +
        df['FLAG_WORK_PHONE'] + df['FLAG_CONT_MOBILE'] +
        df['FLAG_PHONE'] + df['FLAG_EMAIL']
    )
    df['HAS_EMAIL'] = df['FLAG_EMAIL']
    df['HAS_WORK_PHONE'] = df['FLAG_WORK_PHONE']

    # Document flags aggregation
    doc_cols = [col for col in df.columns if 'FLAG_DOCUMENT_' in col]
    df['DOCUMENT_COUNT'] = df[doc_cols].sum(axis=1)
    df['HAS_DOCUMENT'] = df[doc_cols].max(axis=1)
    df['NEW_DOC_IND_AVG'] = df[doc_cols].mean(axis=1)
    df['NEW_DOC_IND_STD'] = df[doc_cols].std(axis=1)
    df = df.drop(columns=doc_cols)

    # Credit bureau enquiries
    bureau_cols = ['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY',
                   'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON',
                   'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']
    df['TOTAL_ENQUIRIES_CREDIT_BUREAU'] = df[bureau_cols].sum(axis=1)
    df['PCTG_ENQUIRIES_WEEK'] = safe_divide(
        df['AMT_REQ_CREDIT_BUREAU_WEEK'],
        df['TOTAL_ENQUIRIES_CREDIT_BUREAU']
    )
    df['PCTG_ENQUIRIES_MON'] = safe_divide(
        df['AMT_REQ_CREDIT_BUREAU_MON'],
        df['TOTAL_ENQUIRIES_CREDIT_BUREAU']
    )
    df['PCTG_ENQUIRIES_QRT'] = safe_divide(
        df['AMT_REQ_CREDIT_BUREAU_QRT'],
        df['TOTAL_ENQUIRIES_CREDIT_BUREAU']
    )
    df['PCTG_ENQUIRIES_YEAR'] = safe_divide(
        df['AMT_REQ_CREDIT_BUREAU_YEAR'],
        df['TOTAL_ENQUIRIES_CREDIT_BUREAU']
    )
    df = df.drop(columns=['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY',
                          'AMT_REQ_CREDIT_BUREAU_WEEK'])

    # -------------------------------------------------------------------------
    # 1.3. RATIOS - Core Credit Scoring Features
    # -------------------------------------------------------------------------

    # Income ratios
    df['ANNUITY_INCOME_PERC'] = safe_divide(df['AMT_ANNUITY'], df['AMT_INCOME_TOTAL'])
    df['CREDIT_INCOME_PERC'] = safe_divide(df['AMT_CREDIT'], df['AMT_INCOME_TOTAL'])
    df['income_credit_percentage'] = safe_divide(df['AMT_INCOME_TOTAL'], df['AMT_CREDIT'])

    # Payment ability
    df['PAYMENT_RATE'] = safe_divide(df['AMT_ANNUITY'], df['AMT_CREDIT'])
    df['credit_to_annuity_ratio'] = safe_divide(df['AMT_CREDIT'], df['AMT_ANNUITY'])

    # Per person ratios
    df['income_per_person'] = safe_divide(df['AMT_INCOME_TOTAL'], df['CNT_FAM_MEMBERS'])
    df['income_per_child'] = safe_divide(
        df['AMT_INCOME_TOTAL'],
        df['CNT_CHILDREN'] + 1
    ) * (df['CNT_CHILDREN'] > 0)
    df['income_per_non_child'] = safe_divide(
        df['AMT_INCOME_TOTAL'],
        df['cnt_non_child']
    )

    df['credit_per_person'] = safe_divide(df['AMT_CREDIT'], df['CNT_FAM_MEMBERS'])
    df['credit_per_child'] = safe_divide(df['AMT_CREDIT'], df['CNT_CHILDREN'] + 1)
    df['credit_per_non_child'] = safe_divide(df['AMT_CREDIT'], df['cnt_non_child'])

    # Goods ratios
    df['credit_to_goods_ratio'] = safe_divide(df['AMT_CREDIT'], df['AMT_GOODS_PRICE'])
    df['CREDIT_GOODS_PRICE_RATIO1'] = safe_divide(
        df['AMT_CREDIT'] - df['AMT_GOODS_PRICE'],
        df['AMT_GOODS_PRICE']
    )
    df['CREDIT_GOODS_PRICE_RATIO2'] = safe_divide(
        df['AMT_CREDIT'] - df['AMT_GOODS_PRICE'],
        df['AMT_CREDIT']
    )
    df['CREDIT_DOWN_PAYMENT'] = df['AMT_GOODS_PRICE'] - df['AMT_CREDIT']

    # Time-based ratios
    df['DAYS_EMPLOYED_FILLED'] = df['DAYS_EMPLOYED'].fillna(0)
    df['days_employed_percentage'] = safe_divide(
        df['DAYS_EMPLOYED_FILLED'],
        df['DAYS_BIRTH']
    )
    df['car_to_birth_ratio'] = RELU(safe_divide(df['OWN_CAR_AGE'], df['DAYS_BIRTH']))
    df['car_to_employ_ratio'] = RELU(safe_divide(
        df['OWN_CAR_AGE'],
        df['DAYS_EMPLOYED_FILLED']
    ))
    df['phone_to_birth_ratio'] = RELU(safe_divide(
        df['DAYS_LAST_PHONE_CHANGE'],
        df['DAYS_BIRTH']
    ))
    df['phone_to_employ_ratio'] = RELU(safe_divide(
        df['DAYS_LAST_PHONE_CHANGE'],
        df['DAYS_EMPLOYED_FILLED']
    ))

    # Debt burden
    df['DEBT_BURDEN_PER_WORKING_DAY'] = safe_divide(
        df['PAYMENT_RATE'],
        df['DAYS_EMPLOYED_FILLED']
    )
    df['DEBT_BURDEN_PER_LIFE_DAY'] = safe_divide(df['PAYMENT_RATE'], df['DAYS_BIRTH'])

    # Building ratios
    df['LIVINGAREA_AVG'] = df['LIVINGAREA_AVG'].fillna(0)
    df['TOTALAREA_MODE'] = df['TOTALAREA_MODE'].fillna(0)
    df['RATIO_AMT_GOODS_PRICE_TO_LIVINGAREA'] = safe_divide(
        df['AMT_GOODS_PRICE'],
        df['LIVINGAREA_AVG'].clip(0.05, 999)
    ) * (df['LIVINGAREA_AVG'] > 0)

    # -------------------------------------------------------------------------
    # 1.4. EXT_SOURCE FEATURES
    # -------------------------------------------------------------------------

    df['ext_sources_mean'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    df['ext_sources_sum'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].sum(axis=1)
    df['ext_sources_var'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].var(axis=1)
    df['ext_sources_weighted'] = (
        df['EXT_SOURCE_1'] * 2 +
        df['EXT_SOURCE_2'] * 3 +
        df['EXT_SOURCE_3'] * 4
    )
    df['EXT_SOURCE_MISSING_VALUES'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].isna().sum(axis=1)

    # -------------------------------------------------------------------------
    # 1.5. TIME TRANSFORMS
    # -------------------------------------------------------------------------

    # Cyclic encoding for hour
    df['sin_HOUR_APPR_PROCESS_START'] = np.sin(2 * np.pi * df['HOUR_APPR_PROCESS_START'] / 24)
    df['cos_HOUR_APPR_PROCESS_START'] = np.cos(2 * np.pi * df['HOUR_APPR_PROCESS_START'] / 24)
    df = df.drop(columns=['HOUR_APPR_PROCESS_START'])

    # Weekday simplification
    df['WEEKDAY_APPR_PROCESS_START'] = df['WEEKDAY_APPR_PROCESS_START'].replace({
        'MONDAY': 'week_day', 'TUESDAY': 'week_day', 'WEDNESDAY': 'week_day',
        'THURSDAY': 'week_day', 'FRIDAY': 'week_day',
        'SATURDAY': 'weekend', 'SUNDAY': 'weekend'
    })

    # Age in years
    df['AGE_INT'] = -df['DAYS_BIRTH'] // 365

    # -------------------------------------------------------------------------
    # 1.6. GROUP-BASED FEATURES
    # -------------------------------------------------------------------------

    # Education mapping
    edu_map = {
        'Lower secondary': 0,
        'Secondary / secondary special': 1,
        'Incomplete higher': 2,
        'Higher education': 3,
        'Academic degree': 5
    }
    df['NAME_EDUCATION_TYPE'] = df['NAME_EDUCATION_TYPE'].map(edu_map).fillna(0).astype(int)

    # Organization simplification
    def group_organizations(org_type):
        if not isinstance(org_type, str):
            return org_type
        if 'Trade' in org_type:
            return np.nan
        elif 'Industry' in org_type:
            return 'Industry'
        elif 'Business' in org_type:
            return 'Business Entity'
        elif 'Transport' in org_type:
            return 'Transport'
        elif 'University' in org_type or 'School' in org_type:
            return 'School'
        elif org_type in ['Unknown', 'XNA', 'Insurance', 'Services', 'Restaurant',
                          'Housing', 'Hotel', 'Agriculture', 'Other']:
            return np.nan
        return org_type

    df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].apply(group_organizations)

    # Group-based income features
    med_income_org_edu = df.groupby(['ORGANIZATION_TYPE', 'NAME_EDUCATION_TYPE'])['AMT_INCOME_TOTAL'].transform('median')
    med_income_org = df.groupby('ORGANIZATION_TYPE')['AMT_INCOME_TOTAL'].transform('median')

    df['income_ratio'] = safe_divide(df['AMT_INCOME_TOTAL'], med_income_org_edu)
    df['income_ratio2'] = safe_divide(df['AMT_INCOME_TOTAL'], med_income_org)
    df['true_annuity_div_income'] = safe_divide(df['AMT_ANNUITY'], med_income_org_edu)
    df['true_annuity_div_income2'] = safe_divide(df['AMT_ANNUITY'], med_income_org)

    # Social circle features
    df['OBS_30_CNT_SOCIAL_CIRCLE'] = df['OBS_30_CNT_SOCIAL_CIRCLE'].clip(0, 50)
    df['DEF_30_CNT_SOCIAL_CIRCLE'] = df['DEF_30_CNT_SOCIAL_CIRCLE'].clip(0, 50)
    df['OBS_60_CNT_SOCIAL_CIRCLE'] = df['OBS_60_CNT_SOCIAL_CIRCLE'].clip(0, 50)
    df['DEF_60_CNT_SOCIAL_CIRCLE'] = df['DEF_60_CNT_SOCIAL_CIRCLE'].clip(0, 50)

    return df


In [ ]:
# ============================================================================
# 2. BUREAU + BUREAU_BALANCE
# ============================================================================

def process_bureau_balance(bureau_balance: pd.DataFrame) -> pd.DataFrame:
    """Xử lý bureau_balance - DPD buckets"""
    bb = bureau_balance.copy()

    # Map STATUS to DPD buckets
    dpd_map = {
        '0': 'no_dpd',
        '1': 'dpd_1_29',
        '2': 'dpd_30_59',
        '3': 'dpd_60_89',
        '4': 'dpd_90_119',
        '5': 'dpd_120_plus',
        'C': 'no_info_or_closed',
        'X': 'no_info_or_closed'
    }
    bb['STATUS_GROUP'] = bb['STATUS'].map(dpd_map)

    # Aggregation per SK_ID_BUREAU
    bb_agg = bb.groupby('SK_ID_BUREAU').agg({
        'MONTHS_BALANCE': 'count',
        'STATUS_GROUP': lambda x: (x == 'no_dpd').sum()
    }).reset_index()
    bb_agg.columns = ['SK_ID_BUREAU', 'BB_MONTHS_COUNT', 'BB_MONTHS_NO_DPD']
    bb_agg['BB_RATIO_NO_DPD'] = bb_agg['BB_MONTHS_NO_DPD'] / bb_agg['BB_MONTHS_COUNT']

    return bb_agg


def process_bureau(bureau_path: str, bureau_balance_path: str = None) -> pd.DataFrame:
    """
    Xử lý bảng bureau với:
    - Cleaning anomalies
    - Debt ratios
    - Aggregation theo CREDIT_ACTIVE
    - Aggregation theo CREDIT_TYPE
    """
    bureau = pd.read_csv(bureau_path)

    # Clean extreme negative values
    for col in ['DAYS_CREDIT_ENDDATE', 'DAYS_CREDIT_UPDATE', 'DAYS_ENDDATE_FACT']:
        bureau.loc[bureau[col] < -40000, col] = np.nan

    # Duration features
    bureau['CREDIT_DURATION'] = bureau['DAYS_CREDIT'] - bureau['DAYS_CREDIT_ENDDATE']
    bureau['ENDDATE_DIF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']

    # Debt & utilization ratios
    bureau['DEBT_PERCENTAGE'] = safe_divide(
        bureau['AMT_CREDIT_SUM'],
        bureau['AMT_CREDIT_SUM_DEBT']
    ).clip(-9, 9)

    bureau['BUREAU_CREDIT_DEBT_RATIO'] = safe_divide(
        bureau['AMT_CREDIT_SUM_DEBT'],
        bureau['AMT_CREDIT_SUM']
    ).clip(-9, 9)

    bureau['CREDIT_TO_ANNUITY_RATIO'] = safe_divide(
        bureau['AMT_CREDIT_SUM'],
        bureau['AMT_ANNUITY']
    ).clip(-9, 9)

    bureau['UTILIZATION_RATIO'] = safe_divide(
        bureau['AMT_CREDIT_SUM_DEBT'],
        bureau['AMT_CREDIT_SUM_LIMIT']
    ).clip(-9, 9)

    bureau['DEBT_CREDIT_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']

    # Time windows
    bureau['LAST_30_DAYS'] = (bureau['DAYS_CREDIT'] >= -30).astype(int)
    bureau['LAST_180_DAYS'] = (bureau['DAYS_CREDIT'] >= -180).astype(int)
    bureau['LAST_365_DAYS'] = (bureau['DAYS_CREDIT'] >= -365).astype(int)
    bureau['LAST_1095_DAYS'] = (bureau['DAYS_CREDIT'] >= -1095).astype(int)
    bureau['LIFE_TIME_CREDIT'] = (bureau['DAYS_CREDIT_ENDDATE'] >= 16000).astype(int)

    # -------------------------------------------------------------------------
    # GENERAL AGGREGATIONS
    # -------------------------------------------------------------------------

    general_agg = {
        'SK_ID_BUREAU': 'count',
        'AMT_CREDIT_SUM': ['sum', 'mean', 'std'],
        'AMT_CREDIT_SUM_DEBT': ['sum', 'mean', 'std'],
        'AMT_CREDIT_SUM_OVERDUE': ['sum', 'mean'],
        'AMT_ANNUITY': ['sum', 'mean', 'std'],
        'AMT_CREDIT_MAX_OVERDUE': ['sum', 'mean', 'std'],
        'AMT_CREDIT_SUM_LIMIT': ['sum', 'mean', 'std'],
        'DAYS_CREDIT': ['min', 'max', 'mean', 'std'],
        'DAYS_CREDIT_UPDATE': ['mean', 'max'],
        'CREDIT_DURATION': ['mean', 'std'],
        'ENDDATE_DIF': 'mean',
        'DEBT_PERCENTAGE': ['mean', 'std'],
        'DEBT_CREDIT_DIFF': ['mean', 'std'],
        'CREDIT_TO_ANNUITY_RATIO': ['mean', 'std'],
        'BUREAU_CREDIT_DEBT_RATIO': ['mean', 'std'],
        'UTILIZATION_RATIO': ['mean', 'std'],
        'CREDIT_TYPE': 'nunique',
        'CREDIT_ACTIVE': 'nunique',
        'CNT_CREDIT_PROLONG': 'mean',
    }

    bureau_agg = bureau.groupby('SK_ID_CURR').agg(general_agg)
    bureau_agg.columns = ['BUREAU_' + '_'.join(col).upper() for col in bureau_agg.columns]
    bureau_agg = bureau_agg.reset_index()

    # Derived feature
    bureau_agg['BUREAU_AVG_PAST_LOAN_PER_TYPE'] = safe_divide(
        bureau_agg['BUREAU_CREDIT_TYPE_NUNIQUE'],
        bureau_agg['BUREAU_SK_ID_BUREAU_COUNT']
    )

    # -------------------------------------------------------------------------
    # BY CREDIT_TYPE
    # -------------------------------------------------------------------------

    credit_types = ['Car loan', 'Credit card', 'Mortgage', 'Consumer credit', 'Microloan']
    for credit_type in credit_types:
        type_df = bureau[bureau['CREDIT_TYPE'] == credit_type]
        if len(type_df) > 0:
            type_agg = type_df.groupby('SK_ID_CURR').agg({
                'AMT_CREDIT_SUM': 'sum',
                'AMT_CREDIT_SUM_DEBT': 'sum',
                'AMT_CREDIT_SUM_OVERDUE': 'sum'
            }).reset_index()

            type_name = credit_type.replace(' ', '_').upper()
            type_agg.columns = ['SK_ID_CURR',
                               f'BUREAU_SUM_{type_name}',
                               f'BUREAU_SUM_DEBT_{type_name}',
                               f'BUREAU_SUM_OVERDUE_{type_name}']
            bureau_agg = bureau_agg.merge(type_agg, on='SK_ID_CURR', how='left')

    # -------------------------------------------------------------------------
    # BY CREDIT_ACTIVE
    # -------------------------------------------------------------------------

    # Active loans
    active = bureau[bureau['CREDIT_ACTIVE'] == 'Active']
    if len(active) > 0:
        active_agg = active.groupby('SK_ID_CURR').agg({
            'SK_ID_BUREAU': 'count',
            'AMT_CREDIT_SUM': 'sum',
            'AMT_CREDIT_SUM_DEBT': 'sum',
            'LAST_30_DAYS': 'sum',
            'LAST_180_DAYS': 'sum',
            'LAST_365_DAYS': 'sum',
            'LIFE_TIME_CREDIT': 'sum'
        }).reset_index()
        active_agg.columns = ['SK_ID_CURR', 'BUREAU_ACTIVE_COUNT',
                             'BUREAU_ACTIVE_SUM_CREDIT', 'BUREAU_ACTIVE_SUM_DEBT',
                             'BUREAU_ACTIVE_LAST_30D', 'BUREAU_ACTIVE_LAST_180D',
                             'BUREAU_ACTIVE_LAST_365D', 'BUREAU_ACTIVE_LIFETIME']
        bureau_agg = bureau_agg.merge(active_agg, on='SK_ID_CURR', how='left')

    # Closed loans
    closed = bureau[bureau['CREDIT_ACTIVE'] == 'Closed']
    if len(closed) > 0:
        closed['LATENCY'] = closed['DAYS_CREDIT_ENDDATE'] - closed['DAYS_ENDDATE_FACT']
        closed_agg = closed.groupby('SK_ID_CURR').agg({
            'SK_ID_BUREAU': 'count',
            'AMT_CREDIT_SUM': 'sum',
            'LATENCY': ['mean', 'sum'],
            'LAST_365_DAYS': 'sum'
        }).reset_index()
        closed_agg.columns = ['SK_ID_CURR', 'BUREAU_CLOSED_COUNT',
                             'BUREAU_CLOSED_SUM_CREDIT',
                             'BUREAU_CLOSED_LATENCY_MEAN',
                             'BUREAU_CLOSED_LATENCY_SUM',
                             'BUREAU_CLOSED_LAST_365D']
        bureau_agg = bureau_agg.merge(closed_agg, on='SK_ID_CURR', how='left')

    # Bad debt
    bad_debt = bureau[bureau['CREDIT_ACTIVE'] == 'Bad debt']
    if len(bad_debt) > 0:
        bad_agg = bad_debt.groupby('SK_ID_CURR').agg({
            'SK_ID_BUREAU': 'count',
            'AMT_CREDIT_SUM': 'sum'
        }).reset_index()
        bad_agg.columns = ['SK_ID_CURR', 'BUREAU_BAD_DEBT_COUNT', 'BUREAU_BAD_DEBT_SUM']
        bureau_agg = bureau_agg.merge(bad_agg, on='SK_ID_CURR', how='left')

    # Overall debt/credit ratio
    bureau_agg['BUREAU_DEBT_CREDIT_RATIO'] = safe_divide(
        bureau_agg['BUREAU_AMT_CREDIT_SUM_DEBT_SUM'],
        bureau_agg['BUREAU_AMT_CREDIT_SUM_SUM']
    )

    # Merge bureau_balance if provided
    if bureau_balance_path:
        bb = pd.read_csv(bureau_balance_path)
        bb_agg = process_bureau_balance(bb)
        bureau = bureau.merge(bb_agg, on='SK_ID_BUREAU', how='left')
        bb_curr_agg = bureau.groupby('SK_ID_CURR').agg({
            'BB_MONTHS_COUNT': 'mean',
            'BB_RATIO_NO_DPD': 'mean'
        }).reset_index()
        bb_curr_agg.columns = ['SK_ID_CURR', 'BUREAU_BB_MONTHS_AVG', 'BUREAU_BB_RATIO_NO_DPD_AVG']
        bureau_agg = bureau_agg.merge(bb_curr_agg, on='SK_ID_CURR', how='left')

    return bureau_agg

In [ ]:
# ============================================================================
# 3. PREVIOUS_APPLICATION
# ============================================================================

def process_previous_application(prev_path: str) -> pd.DataFrame:
    """
    Xử lý previous_application với:
    - Category simplification
    - Domain features (FINISH_RATE, ACTIVE, DURATION)
    - Financial ratios
    - Insurance features
    - Aggregation: general, approved, refused
    """
    prev = pd.read_csv(prev_path)

    # -------------------------------------------------------------------------
    # 3.1. CATEGORY CLEANING
    # -------------------------------------------------------------------------

    # Goods category simplification
    goods_to_other = ['Gardening', 'Animals', 'Insurance', 'Medicine', 'Fitness',
                      'Direct Sales', 'Additional Service', 'Education', 'Weapon']
    prev['NAME_GOODS_CATEGORY'] = prev['NAME_GOODS_CATEGORY'].replace(goods_to_other, 'Other')

    # Cash loan purpose simplification
    purpose_to_other = ['Refusal to name the goal', 'Money for a third person',
                        'Buying a garage', 'Gasification / water supply',
                        'Business development', 'Buying a holiday home / land']
    prev['NAME_CASH_LOAN_PURPOSE'] = prev['NAME_CASH_LOAN_PURPOSE'].replace(purpose_to_other, 'Other')
    prev['NAME_CASH_LOAN_PURPOSE'] = prev['NAME_CASH_LOAN_PURPOSE'].replace(
        ['Buying a new car', 'Buying a used car'], 'Buying a car'
    )

    # Boolean mapping
    prev['FLAG_LAST_APPL_PER_CONTRACT'] = prev['FLAG_LAST_APPL_PER_CONTRACT'].map({'Y': 1, 'N': 0})

    # Type suite simplification
    prev['NAME_TYPE_SUITE'] = prev['NAME_TYPE_SUITE'].replace({
        'Children': 'Family',
        'Spouse, partner': 'Family',
        'Other_A': 'Other',
        'Other_B': 'Other',
        'Unaccompanied': 'Single'
    })

    # Yield group mapping
    prev['NAME_YIELD_GROUP'] = prev['NAME_YIELD_GROUP'].replace({
        'XNA': 0, 'low_action': 1, 'low_normal': 1, 'middle': 3, 'high': 4
    }).fillna(0)

    # -------------------------------------------------------------------------
    # 3.2. DOMAIN FEATURES
    # -------------------------------------------------------------------------

    # Time-based features
    prev['ACTIVE'] = (prev['DAYS_LAST_DUE'] > 0).astype(int)
    prev['ACTUAL_DURATION'] = prev['DAYS_LAST_DUE'] - prev['DAYS_FIRST_DUE']
    prev['DURATION'] = prev['DAYS_TERMINATION'] - prev['DAYS_FIRST_DUE']
    prev['LIFETIME_LOAN'] = (prev['DAYS_TERMINATION'] > 16000).astype(int)

    # Finish rate (completion ratio)
    prev['FINISH_RATE'] = safe_divide(
        prev['ACTUAL_DURATION'],
        prev['DURATION']
    ) * (1 - prev['ACTIVE']) * (1 - prev['LIFETIME_LOAN'])

    # -------------------------------------------------------------------------
    # 3.3. FINANCIAL RATIOS
    # -------------------------------------------------------------------------

    prev['AMT_DOWN_PAYMENT'] = prev['AMT_DOWN_PAYMENT'].fillna(0)

    # Application vs credit
    prev['APP_CREDIT_PERC'] = safe_divide(
        prev['AMT_APPLICATION'],
        prev['AMT_CREDIT']
    )

    # Down payment & goods ratios
    prev['DOWN_PAYMENT_RATIO'] = safe_divide(
        prev['AMT_DOWN_PAYMENT'],
        prev['AMT_CREDIT']
    )
    prev['GOODS_PRICE_RATIO'] = safe_divide(
        prev['AMT_GOODS_PRICE'],
        prev['AMT_CREDIT']
    )

    # Credit to annuity
    prev['CREDIT_ANNUITY_RATIO'] = safe_divide(
        prev['AMT_CREDIT'],
        prev['AMT_ANNUITY']
    )
    prev['RATIO_APPLICATION_TO_ANNUITY'] = safe_divide(
        prev['AMT_APPLICATION'],
        prev['AMT_ANNUITY']
    )
    prev['RATIO_GOODS_TO_ANNUITY'] = safe_divide(
        prev['AMT_GOODS_PRICE'],
        prev['AMT_ANNUITY']
    )

    # Credit vs goods
    prev['CREDIT_DOWNPAYMENT'] = prev['AMT_GOODS_PRICE'] - prev['AMT_DOWN_PAYMENT']

    # Difference tracking
    prev['AMT_DIFF_PERCENT'] = safe_divide(
        prev['AMT_CREDIT'] - prev['AMT_APPLICATION'],
        prev['AMT_APPLICATION']
    ) * 100

    # -------------------------------------------------------------------------
    # 3.4. INSURANCE FEATURES
    # -------------------------------------------------------------------------

    prev['Insuranced_amt_application'] = prev['AMT_APPLICATION'] * prev['NFLAG_INSURED_ON_APPROVAL']
    prev['Insuranced_amt_credit'] = prev['AMT_CREDIT'] * prev['NFLAG_INSURED_ON_APPROVAL']
    prev['HIGH_AMT_APPLICATION'] = (prev['AMT_APPLICATION'] > 100000).astype(int)
    prev['HIGH_DOWN_PAYMENT'] = (prev['AMT_DOWN_PAYMENT'] > 10000).astype(int)

    # -------------------------------------------------------------------------
    # 3.5. INTEREST CALCULATION
    # -------------------------------------------------------------------------

    prev['INTEREST'] = prev['CNT_PAYMENT'] * prev['AMT_ANNUITY'] - prev['AMT_CREDIT']
    prev['INTEREST_RATE'] = safe_divide(
        2 * 12 * prev['INTEREST'],
        prev['AMT_CREDIT'] * (prev['CNT_PAYMENT'] + 1)
    )
    prev['INTEREST_SHARE'] = safe_divide(prev['INTEREST'], prev['AMT_CREDIT'])

    # -------------------------------------------------------------------------
    # 3.6. CHURN & BEHAVIOR
    # -------------------------------------------------------------------------

    prev['CHURN_PREV'] = (
        prev['DAYS_LAST_DUE_1ST_VERSION'] - prev['DAYS_LAST_DUE']
    ).apply(lambda x: 1 if x >= 0 else (0 if x < 0 else np.nan))

    # Time simplification
    prev['WEEKDAY_APPR_PROCESS_START'] = prev['WEEKDAY_APPR_PROCESS_START'].replace(
        ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY'], 'WEEK_DAY'
    ).replace(['SATURDAY', 'SUNDAY'], 'WEEKEND').fillna('WEEK_DAY')

    # -------------------------------------------------------------------------
    # 3.7. AGGREGATIONS
    # -------------------------------------------------------------------------

    # General aggregation
    general_agg = {
        'AMT_ANNUITY': ['sum', 'min', 'max', 'mean', 'std'],
        'AMT_APPLICATION': ['sum', 'mean', 'std'],
        'AMT_CREDIT': ['sum', 'min', 'max', 'mean', 'std'],
        'APP_CREDIT_PERC': ['mean', 'std'],
        'AMT_DOWN_PAYMENT': ['sum', 'max', 'mean', 'std'],
        'AMT_GOODS_PRICE': ['sum', 'min', 'max', 'mean'],
        'CREDIT_ANNUITY_RATIO': ['mean', 'std'],
        'DOWN_PAYMENT_RATIO': ['mean', 'std'],
        'GOODS_PRICE_RATIO': ['mean', 'std'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
        'HIGH_DOWN_PAYMENT': 'sum',
        'RATIO_APPLICATION_TO_ANNUITY': ['mean', 'std'],
        'RATIO_GOODS_TO_ANNUITY': ['mean', 'std'],
        'INTEREST_RATE': ['mean', 'std'],
        'INTEREST_SHARE': ['mean', 'std'],
    }

    prev_agg = prev.groupby('SK_ID_CURR').agg(general_agg)
    prev_agg.columns = ['PREV_' + '_'.join(col).upper() for col in prev_agg.columns]
    prev_agg = prev_agg.reset_index()

    # Approved loans aggregation
    approved = prev[prev['NAME_CONTRACT_STATUS'] == 'Approved']
    if len(approved) > 0:
        approved_agg = {
            'AMT_ANNUITY': ['sum', 'mean', 'std'],
            'AMT_APPLICATION': ['sum', 'mean'],
            'AMT_CREDIT': ['sum', 'mean', 'std'],
            'APP_CREDIT_PERC': ['mean', 'std'],
            'AMT_DOWN_PAYMENT': ['sum', 'mean'],
            'CREDIT_ANNUITY_RATIO': ['mean', 'std'],
            'CREDIT_DOWNPAYMENT': ['sum', 'mean', 'std'],
            'DOWN_PAYMENT_RATIO': ['mean', 'std', 'max'],
            'NFLAG_INSURED_ON_APPROVAL': 'sum',
            'Insuranced_amt_application': ['sum', 'mean'],
            'Insuranced_amt_credit': ['sum', 'mean'],
            'DAYS_DECISION': 'max',
            'CNT_PAYMENT': ['mean', 'sum'],
            'ACTIVE': 'mean',
            'DURATION': ['sum', 'max', 'min', 'mean', 'std'],
            'ACTUAL_DURATION': ['sum', 'max', 'mean', 'std'],
            'LIFETIME_LOAN': ['sum', 'mean'],
            'FINISH_RATE': ['mean', 'std'],
        }

        app_agg = approved.groupby('SK_ID_CURR').agg(approved_agg)
        app_agg.columns = ['APPROVED_' + '_'.join(col).upper() for col in app_agg.columns]
        app_agg = app_agg.reset_index()

        # Insurance ratios
        app_agg['APPROVED_RATIO_AMT_APPLICATION_INSURANCED'] = safe_divide(
            app_agg['APPROVED_INSURANCED_AMT_APPLICATION_SUM'],
            app_agg['APPROVED_AMT_APPLICATION_SUM']
        )
        app_agg['APPROVED_RATIO_AMT_CREDIT_INSURANCED'] = safe_divide(
            app_agg['APPROVED_INSURANCED_AMT_CREDIT_SUM'],
            app_agg['APPROVED_AMT_CREDIT_SUM']
        )

        prev_agg = prev_agg.merge(app_agg, on='SK_ID_CURR', how='left')

    # Refused loans aggregation
    refused = prev[prev['NAME_CONTRACT_STATUS'] == 'Refused']
    if len(refused) > 0:
        refused_agg = {
            'AMT_ANNUITY': ['sum', 'mean'],
            'AMT_APPLICATION': ['sum', 'mean'],
            'AMT_CREDIT': ['sum', 'mean'],
            'APP_CREDIT_PERC': 'mean',
            'AMT_DOWN_PAYMENT': ['sum', 'mean'],
            'CREDIT_ANNUITY_RATIO': 'mean',
            'DAYS_DECISION': ['max', 'mean'],
        }

        ref_agg = refused.groupby('SK_ID_CURR').agg(refused_agg)
        ref_agg.columns = ['REFUSED_' + '_'.join(col).upper() for col in ref_agg.columns]
        ref_agg = ref_agg.reset_index()

        prev_agg = prev_agg.merge(ref_agg, on='SK_ID_CURR', how='left')

    # Time-based features (last N applications)
    prev_sorted = prev.sort_values(['SK_ID_CURR', 'DAYS_DECISION'])

    # Number of previous applications
    prev_count = prev_sorted.groupby('SK_ID_CURR')['SK_ID_PREV'].count().reset_index()
    prev_count.columns = ['SK_ID_CURR', 'PREV_APPLICATION_COUNT']
    prev_agg = prev_agg.merge(prev_count, on='SK_ID_CURR', how='left')

    # Last application was approved/refused
    prev_sorted['PREV_WAS_APPROVED'] = (prev_sorted['NAME_CONTRACT_STATUS'] == 'Approved').astype(int)
    prev_sorted['PREV_WAS_REFUSED'] = (prev_sorted['NAME_CONTRACT_STATUS'] == 'Refused').astype(int)

    last_status = prev_sorted.groupby('SK_ID_CURR').agg({
        'PREV_WAS_APPROVED': 'last',
        'PREV_WAS_REFUSED': 'last'
    }).reset_index()
    last_status.columns = ['SK_ID_CURR', 'PREV_LAST_WAS_APPROVED', 'PREV_LAST_WAS_REFUSED']
    prev_agg = prev_agg.merge(last_status, on='SK_ID_CURR', how='left')

    # Last N applications features
    for n in [1, 3, 5]:
        tail = prev_sorted.groupby('SK_ID_CURR').tail(n)
        tail_agg = tail.groupby('SK_ID_CURR').agg({
            'CNT_PAYMENT': 'mean',
            'DAYS_DECISION': 'mean',
            'DAYS_FIRST_DRAWING': 'mean'
        }).reset_index()
        tail_agg.columns = ['SK_ID_CURR',
                           f'PREV_LAST_{n}_CNT_PAYMENT_MEAN',
                           f'PREV_LAST_{n}_DAYS_DECISION_MEAN',
                           f'PREV_LAST_{n}_DAYS_FIRST_DRAWING_MEAN']
        prev_agg = prev_agg.merge(tail_agg, on='SK_ID_CURR', how='left')

    return prev_agg

In [ ]:
# ============================================================================
# 4. POS_CASH_BALANCE
# ============================================================================

def process_pos_cash(pos_path: str) -> pd.DataFrame:
    """
    Xử lý POS_CASH_balance với:
    - Status flags
    - DPD tracking
    - Time windows
    - Long/short term separation
    """
    pos = pd.read_csv(pos_path)

    # Remove invalid statuses
    pos = pos[~pos['NAME_CONTRACT_STATUS'].isin(['XNA', 'Canceled'])]

    # -------------------------------------------------------------------------
    # 4.1. STATUS FLAGS
    # -------------------------------------------------------------------------

    pos['COMPLETED'] = (pos['NAME_CONTRACT_STATUS'] == 'Completed').astype(int)
    pos['COUNT_DEMAND'] = (pos['NAME_CONTRACT_STATUS'] == 'Demand').astype(int)
    pos['COUNT_SIGNED'] = (pos['NAME_CONTRACT_STATUS'] == 'Signed').astype(int)
    pos['COUNT_APPROVED'] = (pos['NAME_CONTRACT_STATUS'] == 'Approved').astype(int)

    # -------------------------------------------------------------------------
    # 4.2. DPD FEATURES
    # -------------------------------------------------------------------------

    pos['COUNT_SK_DPD'] = (pos['SK_DPD'] > 0).astype(int)
    pos['COUNT_SK_DPD_DEF'] = (pos['SK_DPD_DEF'] > 0).astype(int)
    pos['MEAN_SK_DPD'] = pos['SK_DPD']
    pos['MEAN_SK_DPD_DEF'] = pos['SK_DPD_DEF']

    # -------------------------------------------------------------------------
    # 4.3. TIME WINDOWS
    # -------------------------------------------------------------------------

    pos['LAST_3_MONTHS'] = (pos['MONTHS_BALANCE'] >= -3).astype(int)
    pos['LAST_6_MONTHS'] = (pos['MONTHS_BALANCE'] >= -6).astype(int)
    pos['LAST_12_MONTHS'] = (pos['MONTHS_BALANCE'] >= -12).astype(int)
    pos['LAST_36_MONTHS'] = (pos['MONTHS_BALANCE'] >= -36).astype(int)

    # -------------------------------------------------------------------------
    # 4.4. AGGREGATION PER SK_ID_PREV
    # -------------------------------------------------------------------------

    pos['NUM_INSTALMENT'] = 1

    pos_prev_agg = pos.groupby('SK_ID_PREV').agg({
        'SK_ID_CURR': 'first',
        'NUM_INSTALMENT': 'sum',
        'COMPLETED': 'sum',
        'COUNT_DEMAND': 'sum',
        'COUNT_SIGNED': 'sum',
        'CNT_INSTALMENT': 'max',
        'MONTHS_BALANCE': 'max',
        'COUNT_SK_DPD': 'sum',
        'COUNT_SK_DPD_DEF': 'sum',
        'SK_DPD': 'sum',
        'SK_DPD_DEF': 'sum',
        'MEAN_SK_DPD': 'mean',
        'MEAN_SK_DPD_DEF': 'mean',
        'CNT_INSTALMENT_FUTURE': 'min',
        'LAST_12_MONTHS': 'sum',
        'LAST_36_MONTHS': 'sum'
    }).reset_index()

    # Long term vs short term
    pos_prev_agg['LONG_TERM'] = (pos_prev_agg['CNT_INSTALMENT'] > 24).astype(int)
    pos_prev_agg['SHORT_TERM'] = (pos_prev_agg['CNT_INSTALMENT'] <= 24).astype(int)

    # -------------------------------------------------------------------------
    # 4.5. AGGREGATION PER SK_ID_CURR
    # -------------------------------------------------------------------------

    # General aggregation
    pos_agg = pos_prev_agg.groupby('SK_ID_CURR').agg({
        'NUM_INSTALMENT': 'sum',
        'COMPLETED': ['sum', 'mean'],
        'COUNT_SIGNED': 'sum',
        'CNT_INSTALMENT': ['sum', 'mean', 'std'],
        'MONTHS_BALANCE': 'max',
        'SK_DPD': ['sum', 'mean', 'max'],
        'SK_DPD_DEF': ['sum', 'mean', 'max'],
        'COUNT_SK_DPD': ['sum', 'mean'],
        'COUNT_SK_DPD_DEF': ['sum', 'mean'],
        'CNT_INSTALMENT_FUTURE': 'sum',
        'LAST_12_MONTHS': 'sum',
        'LAST_36_MONTHS': 'sum'
    })

    # Flatten column names
    pos_agg.columns = ['POS_' + '_'.join(col).upper() for col in pos_agg.columns]
    pos_agg = pos_agg.reset_index()

    # Completion ratio
    pos_agg['POS_COMPLETION_RATIO'] = safe_divide(
        pos_agg['POS_COMPLETED_SUM'],
        pos_agg['POS_NUM_INSTALMENT_SUM']
    )

    # Long-term contracts
    long_term = pos_prev_agg[pos_prev_agg['LONG_TERM'] == 1]
    if len(long_term) > 0:
        lt_agg = long_term.groupby('SK_ID_CURR').agg({
            'CNT_INSTALMENT': 'mean',
            'SK_ID_PREV': 'count',
            'NUM_INSTALMENT': 'sum',
            'LAST_36_MONTHS': 'sum'
        }).reset_index()
        lt_agg.columns = ['SK_ID_CURR', 'POS_LONG_TERM_CNT_INSTALMENT_MEAN',
                         'POS_LONG_TERM_COUNT', 'POS_LONG_TERM_NUM_INSTALMENT',
                         'POS_LONG_TERM_LAST_36M']
        pos_agg = pos_agg.merge(lt_agg, on='SK_ID_CURR', how='left')

    # Short-term contracts
    short_term = pos_prev_agg[pos_prev_agg['SHORT_TERM'] == 1]
    if len(short_term) > 0:
        st_agg = short_term.groupby('SK_ID_CURR').agg({
            'CNT_INSTALMENT': 'mean',
            'SK_ID_PREV': 'count',
            'NUM_INSTALMENT': 'sum',
            'LAST_12_MONTHS': 'sum',
            'SK_DPD': 'sum'
        }).reset_index()
        st_agg.columns = ['SK_ID_CURR', 'POS_SHORT_TERM_CNT_INSTALMENT_MEAN',
                         'POS_SHORT_TERM_COUNT', 'POS_SHORT_TERM_NUM_INSTALMENT',
                         'POS_SHORT_TERM_LAST_12M', 'POS_SHORT_TERM_SK_DPD']
        pos_agg = pos_agg.merge(st_agg, on='SK_ID_CURR', how='left')

    return pos_agg



In [ ]:
# ============================================================================
# 5. INSTALLMENTS_PAYMENTS
# ============================================================================

def process_installments(inst_path: str) -> pd.DataFrame:
    """
    Xử lý installments_payments với:
    - Payment difference tracking
    - Late/early payment analysis
    - Time window aggregations
    """
    inst = pd.read_csv(inst_path)

    # -------------------------------------------------------------------------
    # 5.1. ROW-LEVEL FEATURES
    # -------------------------------------------------------------------------

    inst['INSTALLMENT_PAYMENT_DIFF'] = inst['AMT_INSTALMENT'] - inst['AMT_PAYMENT']
    inst['DIFF'] = inst['DAYS_INSTALMENT'] - inst['DAYS_ENTRY_PAYMENT']

    # Late/early indicators
    inst['LATE'] = inst['DIFF'].clip(lower=0)
    inst['EARLY'] = (-inst['DIFF']).clip(lower=0)

    # -------------------------------------------------------------------------
    # 5.2. TIME WINDOWS
    # -------------------------------------------------------------------------

    for days in [30, 60, 90, 180, 365, 730]:
        inst[f'LAST_{days}_DAYS'] = (inst['DAYS_ENTRY_PAYMENT'] >= -days).astype(int)

    # -------------------------------------------------------------------------
    # 5.3. WEIGHTED FEATURES BY TIME WINDOW
    # -------------------------------------------------------------------------

    for days in [30, 90, 180, 365, 730]:
        inst[f'PAYMENT_LAST_{days}_DAYS'] = inst[f'LAST_{days}_DAYS'] * inst['AMT_PAYMENT']
        inst[f'LATENESS_LAST_{days}_DAYS'] = inst[f'LAST_{days}_DAYS'] * inst['LATE']
        inst[f'EARLYNESS_LAST_{days}_DAYS'] = inst[f'LAST_{days}_DAYS'] * inst['EARLY']
        inst[f'INSTALLMENT_PAYMENT_DIFF_LAST_{days}_DAYS'] = (
            inst[f'LAST_{days}_DAYS'] * inst['INSTALLMENT_PAYMENT_DIFF']
        )

    # -------------------------------------------------------------------------
    # 5.4. AGGREGATION PER SK_ID_CURR
    # -------------------------------------------------------------------------

    inst_agg_dict = {
        'NUM_INSTALMENT_NUMBER': 'max',
        'SK_ID_PREV': 'nunique',
        'DIFF': ['sum', 'mean', 'std'],
        'LATE': ['sum', 'mean', 'std', 'max'],
        'EARLY': ['sum', 'mean', 'std'],
        'DAYS_ENTRY_PAYMENT': 'max',
        'INSTALLMENT_PAYMENT_DIFF': ['sum', 'mean', 'std'],
        'AMT_PAYMENT': 'sum',
    }

    # Add time window features
    for days in [30, 90, 180, 365, 730]:
        inst_agg_dict[f'LAST_{days}_DAYS'] = 'sum'
        inst_agg_dict[f'PAYMENT_LAST_{days}_DAYS'] = ['sum', 'mean', 'std']
        inst_agg_dict[f'LATENESS_LAST_{days}_DAYS'] = ['sum', 'mean']
        inst_agg_dict[f'EARLYNESS_LAST_{days}_DAYS'] = ['sum', 'mean']
        inst_agg_dict[f'INSTALLMENT_PAYMENT_DIFF_LAST_{days}_DAYS'] = ['sum', 'mean']

    inst_agg = inst.groupby('SK_ID_CURR').agg(inst_agg_dict)

    # Flatten column names properly
    new_cols = []
    for col in inst_agg.columns:
        if isinstance(col, tuple):
            new_cols.append('INST_' + '_'.join(str(x).upper() for x in col))
        else:
            new_cols.append('INST_' + str(col).upper())

    inst_agg.columns = new_cols
    inst_agg = inst_agg.reset_index()

    # Average installment amount
    inst_agg['INST_AVG_PAYMENT_PER_INSTALMENT'] = safe_divide(
        inst_agg['INST_AMT_PAYMENT_SUM'],
        inst_agg['INST_NUM_INSTALMENT_NUMBER_MAX']
    )

    # Late payment rate
    inst_agg['INST_LATE_PAYMENT_RATE'] = safe_divide(
        inst_agg['INST_LATE_SUM'],
        inst_agg['INST_NUM_INSTALMENT_NUMBER_MAX']
    )

    return inst_agg


In [ ]:
# ============================================================================
# 6. CREDIT_CARD_BALANCE
# ============================================================================

def process_credit_card(cc_path: str) -> pd.DataFrame:
    """
    Xử lý credit_card_balance với:
    - Utilization tracking
    - Payment behavior
    - Drawing patterns
    - Time window analysis
    """
    cc = pd.read_csv(cc_path)

    # Clean negative values
    cc.loc[cc['AMT_DRAWINGS_ATM_CURRENT'] < 0, 'AMT_DRAWINGS_ATM_CURRENT'] = np.nan
    cc.loc[cc['AMT_DRAWINGS_CURRENT'] < 0, 'AMT_DRAWINGS_CURRENT'] = np.nan

    # -------------------------------------------------------------------------
    # 6.1. STATUS FLAGS
    # -------------------------------------------------------------------------

    cc['ACTIVE'] = (cc['NAME_CONTRACT_STATUS'] == 'Active').astype(int)
    cc['COMPLETED'] = (cc['NAME_CONTRACT_STATUS'] == 'Completed').astype(int)

    # -------------------------------------------------------------------------
    # 6.2. UTILIZATION & PAYMENT BEHAVIOR
    # -------------------------------------------------------------------------

    cc['UTILIZATION'] = safe_divide(
        cc['AMT_BALANCE'],
        cc['AMT_CREDIT_LIMIT_ACTUAL']
    ).clip(-9, 9)

    cc['RATE_OF_PAYBACK'] = safe_divide(
        cc['AMT_PAYMENT_TOTAL_CURRENT'],
        cc['AMT_INST_MIN_REGULARITY']
    ).clip(-9, 9)

    cc['PERCENTAGE_OF_MINIMUM_PAYMENTS_MISSED'] = safe_divide(
        cc['AMT_PAYMENT_CURRENT'],
        cc['AMT_INST_MIN_REGULARITY']
    ).clip(-9, 9)

    cc['MINIMUM_PAYMENTS_ONLY'] = (
        cc['AMT_PAYMENT_CURRENT'] == cc['AMT_INST_MIN_REGULARITY']
    ).astype(int)

    # -------------------------------------------------------------------------
    # 6.3. DRAWING BEHAVIOR
    # -------------------------------------------------------------------------

    cc['SUM_ALL_CNT_DRAWINGS'] = (
        cc['CNT_DRAWINGS_ATM_CURRENT'] +
        cc['CNT_DRAWINGS_CURRENT'] +
        cc['CNT_DRAWINGS_OTHER_CURRENT'] +
        cc['CNT_DRAWINGS_POS_CURRENT']
    )

    cc['SUM_ALL_AMT_DRAWINGS'] = (
        cc['AMT_DRAWINGS_ATM_CURRENT'].fillna(0) +
        cc['AMT_DRAWINGS_CURRENT'].fillna(0) +
        cc['AMT_DRAWINGS_OTHER_CURRENT'].fillna(0) +
        cc['AMT_DRAWINGS_POS_CURRENT'].fillna(0)
    )

    cc['RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS'] = safe_divide(
        cc['SUM_ALL_AMT_DRAWINGS'],
        cc['SUM_ALL_CNT_DRAWINGS']
    ).clip(-9, 9)

    # -------------------------------------------------------------------------
    # 6.4. FLAGS
    # -------------------------------------------------------------------------

    cc['POSITIVE_CREDIT'] = (cc['AMT_BALANCE'] > 0).astype(int)
    cc['FLAG_UTILIZATION_LESS_50'] = (cc['UTILIZATION'] < 0.50).astype(int)
    cc['FLAG_UTILIZATION_LESS_75'] = (cc['UTILIZATION'] < 0.75).astype(int)
    cc['FLAG_UTILIZATION_MORE_100'] = (cc['UTILIZATION'] > 1.00).astype(int)

    # -------------------------------------------------------------------------
    # 6.5. TIME WINDOWS
    # -------------------------------------------------------------------------

    for months in [6, 12, 36]:
        cc[f'FLAG_{months}_MONTHS'] = (cc['MONTHS_BALANCE'] >= -months).astype(int)

    # -------------------------------------------------------------------------
    # 6.6. AGGREGATIONS
    # -------------------------------------------------------------------------

    # General aggregation
    cc_agg_dict = {
        'MONTHS_BALANCE': 'count',
        'ACTIVE': 'sum',
        'AMT_BALANCE': 'sum',
        'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'max', 'min', 'std'],
        'AMT_INST_MIN_REGULARITY': ['mean', 'max', 'std'],
        'AMT_PAYMENT_TOTAL_CURRENT': ['sum', 'mean', 'max', 'min', 'std'],
        'UTILIZATION': 'sum',
        'FLAG_UTILIZATION_LESS_50': ['sum', 'mean'],
        'FLAG_UTILIZATION_LESS_75': ['sum', 'mean'],
        'FLAG_UTILIZATION_MORE_100': ['sum', 'mean'],
        'CNT_DRAWINGS_ATM_CURRENT': ['mean', 'sum'],
        'CNT_DRAWINGS_CURRENT': ['mean', 'sum'],
        'AMT_DRAWINGS_ATM_CURRENT': ['sum', 'mean', 'std'],
        'AMT_DRAWINGS_CURRENT': ['sum', 'mean', 'max', 'std'],
        'AMT_DRAWINGS_POS_CURRENT': ['sum', 'mean', 'std'],
        'POSITIVE_CREDIT': 'sum',
        'MINIMUM_PAYMENTS_ONLY': 'sum',
        'SUM_ALL_CNT_DRAWINGS': ['sum', 'mean'],
        'SUM_ALL_AMT_DRAWINGS': ['sum', 'mean'],
        'RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS': ['mean', 'std'],
        'RATE_OF_PAYBACK': 'sum',
        'PERCENTAGE_OF_MINIMUM_PAYMENTS_MISSED': 'sum',
    }

    cc_agg = cc.groupby('SK_ID_CURR').agg(cc_agg_dict)

    # Flatten column names
    cc_agg.columns = ['CC_' + '_'.join(col).upper() for col in cc_agg.columns]
    cc_agg = cc_agg.reset_index()

    # Normalize by positive credit months
    cc_agg['CC_AMT_BALANCE_AVG'] = safe_divide(
        cc_agg['CC_AMT_BALANCE_SUM'],
        cc_agg['CC_POSITIVE_CREDIT_SUM']
    )
    cc_agg['CC_UTILIZATION_AVG'] = safe_divide(
        cc_agg['CC_UTILIZATION_SUM'],
        cc_agg['CC_POSITIVE_CREDIT_SUM']
    )
    cc_agg['CC_POSITIVE_PERCENT'] = safe_divide(
        cc_agg['CC_POSITIVE_CREDIT_SUM'],
        cc_agg['CC_MONTHS_BALANCE_COUNT']
    )

    # Time window aggregations
    for months in [6, 12, 36]:
        window_df = cc[cc[f'FLAG_{months}_MONTHS'] == 1]
        if len(window_df) > 0:
            window_agg = window_df.groupby('SK_ID_CURR').agg({
                'AMT_BALANCE': 'sum',
                'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'std'],
                'AMT_PAYMENT_TOTAL_CURRENT': 'sum',
                'UTILIZATION': 'sum',
                'CNT_DRAWINGS_CURRENT': 'sum',
                'AMT_DRAWINGS_CURRENT': ['sum', 'max'],
                'POSITIVE_CREDIT': 'sum'
            })

            # Flatten columns
            window_agg.columns = [f'CC_{months}M_' + '_'.join(col).upper()
                                 for col in window_agg.columns]
            window_agg = window_agg.reset_index()

            # Normalize
            window_agg[f'CC_{months}M_AMT_BALANCE_AVG'] = safe_divide(
                window_agg[f'CC_{months}M_AMT_BALANCE_SUM'],
                window_agg[f'CC_{months}M_POSITIVE_CREDIT_SUM']
            )
            window_agg[f'CC_{months}M_UTILIZATION_AVG'] = safe_divide(
                window_agg[f'CC_{months}M_UTILIZATION_SUM'],
                window_agg[f'CC_{months}M_POSITIVE_CREDIT_SUM']
            )

            cc_agg = cc_agg.merge(window_agg, on='SK_ID_CURR', how='left')

    return cc_agg


In [ ]:
# ============================================================================
# 7. MAIN PIPELINE - MERGE ALL
# ============================================================================

def feature_engineering_pipeline(
    app_train_path: str,
    app_test_path: str,
    bureau_path: str,
    bureau_balance_path: str,
    prev_path: str,
    pos_path: str,
    inst_path: str,
    cc_path: str,
    output_dir: str
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.Series]:
    """
    Pipeline tổng thể:
    1. Load và process từng bảng
    2. Merge theo thứ tự
    3. Xử lý NaN
    4. Export X_train_fe, X_test_fe
    """

    # -------------------------------------------------------------------------
    # 7.1. LOAD & PROCESS APPLICATION
    # -------------------------------------------------------------------------

    print("\n[1/8] Processing application_train...")
    app_train = pd.read_csv(app_train_path)
    app_train_fe = process_application(app_train)
    y_train = app_train_fe['TARGET']
    print(f"  Shape: {app_train_fe.shape}")

    print("\n[2/8] Processing application_test...")
    app_test = pd.read_csv(app_test_path)
    app_test_fe = process_application(app_test)
    print(f"  Shape: {app_test_fe.shape}")

    # -------------------------------------------------------------------------
    # 7.2. PROCESS AUXILIARY TABLES
    # -------------------------------------------------------------------------

    print("\n[3/8] Processing bureau + bureau_balance...")
    bureau_agg = process_bureau(bureau_path, bureau_balance_path)
    print(f"  Shape: {bureau_agg.shape}")

    print("\n[4/8] Processing previous_application...")
    prev_agg = process_previous_application(prev_path)
    print(f"  Shape: {prev_agg.shape}")

    print("\n[5/8] Processing POS_CASH_balance...")
    pos_agg = process_pos_cash(pos_path)
    print(f"  Shape: {pos_agg.shape}")

    print("\n[6/8] Processing installments_payments...")
    inst_agg = process_installments(inst_path)
    print(f"  Shape: {inst_agg.shape}")

    print("\n[7/8] Processing credit_card_balance...")
    cc_agg = process_credit_card(cc_path)
    print(f"  Shape: {cc_agg.shape}")

    # -------------------------------------------------------------------------
    # 7.3. MERGE ALL - LEFT JOIN
    # -------------------------------------------------------------------------

    print("\n[8/8] Merging all tables...")

    # Merge train
    print("  Merging train dataset...")
    X_train_fe = app_train_fe.copy()
    X_train_fe = X_train_fe.merge(bureau_agg, on='SK_ID_CURR', how='left')
    print(f"    After bureau: {X_train_fe.shape}")

    X_train_fe = X_train_fe.merge(prev_agg, on='SK_ID_CURR', how='left')
    print(f"    After previous: {X_train_fe.shape}")

    X_train_fe = X_train_fe.merge(pos_agg, on='SK_ID_CURR', how='left')
    print(f"    After POS: {X_train_fe.shape}")

    X_train_fe = X_train_fe.merge(inst_agg, on='SK_ID_CURR', how='left')
    print(f"    After installments: {X_train_fe.shape}")

    X_train_fe = X_train_fe.merge(cc_agg, on='SK_ID_CURR', how='left')
    print(f"    After credit card: {X_train_fe.shape}")

    # Merge test
    print("  Merging test dataset...")
    X_test_fe = app_test_fe.copy()
    X_test_fe = X_test_fe.merge(bureau_agg, on='SK_ID_CURR', how='left')
    X_test_fe = X_test_fe.merge(prev_agg, on='SK_ID_CURR', how='left')
    X_test_fe = X_test_fe.merge(pos_agg, on='SK_ID_CURR', how='left')
    X_test_fe = X_test_fe.merge(inst_agg, on='SK_ID_CURR', how='left')
    X_test_fe = X_test_fe.merge(cc_agg, on='SK_ID_CURR', how='left')
    print(f"    Test shape: {X_test_fe.shape}")

    # -------------------------------------------------------------------------
    # 7.4. NaN HANDLING
    # -------------------------------------------------------------------------

    print("\n[Post-processing] Handling missing values...")

    # Drop TARGET from train features
    X_train_fe = X_train_fe.drop(columns=['TARGET'])
    app_cols = set(app_train.columns) - {'TARGET'}
    auxiliary_cols = [col for col in X_train_fe.columns
                     if col not in app_cols and col != 'SK_ID_CURR']

    print(f"  Filling {len(auxiliary_cols)} auxiliary columns with 0...")
    for col in auxiliary_cols:
        X_train_fe[col] = X_train_fe[col].fillna(0)
        X_test_fe[col] = X_test_fe[col].fillna(0)

    # -------------------------------------------------------------------------
    # 7.5. EXPORT
    # -------------------------------------------------------------------------

    print("\n[Export] Saving files...")
    os.makedirs(output_dir, exist_ok=True)

    train_path = os.path.join(output_dir, 'X_train_fe.parquet')
    test_path = os.path.join(output_dir, 'X_test_fe.parquet')
    target_path = os.path.join(output_dir, 'y_train.parquet')

    X_train_fe.to_parquet(train_path, index=False)
    X_test_fe.to_parquet(test_path, index=False)
    y_train.to_frame('TARGET').to_parquet(target_path, index=False)

    print(f"  Saved: {train_path}")
    print(f"  Saved: {test_path}")
    print(f"  Saved: {target_path}")

    # -------------------------------------------------------------------------
    # 7.6. SUMMARY
    # -------------------------------------------------------------------------

    print("FEATURE ENGINEERING COMPLETED")
    print(f"Training set:  {X_train_fe.shape[0]:,} rows × {X_train_fe.shape[1]:,} features")
    print(f"Test set:      {X_test_fe.shape[0]:,} rows × {X_test_fe.shape[1]:,} features")
    print(f"Target:        {y_train.shape[0]:,} samples")
    print(f"Target rate:   {y_train.mean():.2%}")

    # Feature type summary
    numeric_cols = X_train_fe.select_dtypes(include=[np.number]).columns
    categorical_cols = X_train_fe.select_dtypes(exclude=[np.number]).columns

    print(f"\nFeature types:")
    print(f"  Numeric:      {len(numeric_cols):,}")
    print(f"  Categorical:  {len(categorical_cols):,}")

    # NaN summary
    nan_count = X_train_fe.isnull().sum().sum()
    nan_pct = nan_count / (X_train_fe.shape[0] * X_train_fe.shape[1])
    print(f"\nMissing values:")
    print(f"  Total NaN:    {nan_count:,} ({nan_pct:.2%})")

    # Feature groups
    feature_groups = {
        'APP': [c for c in X_train_fe.columns if not any(c.startswith(p) for p in ['BUREAU_', 'PREV_', 'APPROVED_', 'REFUSED_', 'POS_', 'INST_', 'CC_'])],
        'BUREAU': [c for c in X_train_fe.columns if c.startswith('BUREAU_')],
        'PREV': [c for c in X_train_fe.columns if c.startswith(('PREV_', 'APPROVED_', 'REFUSED_'))],
        'POS': [c for c in X_train_fe.columns if c.startswith('POS_')],
        'INST': [c for c in X_train_fe.columns if c.startswith('INST_')],
        'CC': [c for c in X_train_fe.columns if c.startswith('CC_')]
    }

    print(f"\nFeature groups:")
    for group, cols in feature_groups.items():
        print(f"  {group:12s}: {len(cols):4,} features")

    print("=" * 80)

    return X_train_fe, X_test_fe, y_train


In [ ]:
# ============================================================================
# 8. EXECUTION
# ============================================================================

if __name__ == "__main__":

    paths = {
        'app_train': os.path.join(DATA_DIR, "application_train.csv"),
        'app_test': os.path.join(DATA_DIR, "application_test.csv"),
        'bureau': os.path.join(DATA_DIR, "bureau.csv"),
        'bureau_balance': os.path.join(DATA_DIR, "bureau_balance.csv"),
        'prev': os.path.join(DATA_DIR, "previous_application.csv"),
        'pos': os.path.join(DATA_DIR, "POS_CASH_balance.csv"),
        'inst': os.path.join(DATA_DIR, "installments_payments.csv"),
        'cc': os.path.join(DATA_DIR, "credit_card_balance.csv")
    }

    # Run pipeline
    X_train_fe, X_test_fe, y_train = feature_engineering_pipeline(
        app_train_path=paths['app_train'],
        app_test_path=paths['app_test'],
        bureau_path=paths['bureau'],
        bureau_balance_path=paths['bureau_balance'],
        prev_path=paths['prev'],
        pos_path=paths['pos'],
        inst_path=paths['inst'],
        cc_path=paths['cc'],
        output_dir=OUTPUT_DIR
    )

    print("\n✓ Feature engineering completed successfully!")
    print(f"  Files saved to: {OUTPUT_DIR}")

FEATURE ENGINEERING PIPELINE

[1/8] Processing application_train...
  Shape: (307511, 152)

[2/8] Processing application_test...
  Shape: (48744, 151)

[3/8] Processing bureau + bureau_balance...
  Shape: (305811, 74)

[4/8] Processing previous_application...
  Shape: (338857, 112)

[5/8] Processing POS_CASH_balance...
  Shape: (337252, 32)

[6/8] Processing installments_payments...
  Shape: (339587, 70)

[7/8] Processing credit_card_balance...
  Shape: (103558, 83)

[8/8] Merging all tables...
  Merging train dataset...
    After bureau: (307511, 225)
    After previous: (307511, 336)
    After POS: (307511, 367)
    After installments: (307511, 436)
    After credit card: (307511, 518)
  Merging test dataset...
    Test shape: (48744, 517)

[Post-processing] Handling missing values...
  Filling 422 auxiliary columns with 0...

[Export] Saving files...
  Saved: /content/drive/MyDrive/home_credit/processed/X_train_fe.parquet
  Saved: /content/drive/MyDrive/home_credit/processed/X_test_

Tổng quan kết quả:

- Training: 307,511 samples × 517 features
- Test: 48,744 samples × 517 features
- Target rate: 8.07% (imbalanced - cần xử lý khi modeling)